In [1]:
import pandas as pd
import numpy as np

In [20]:
# data separation 
d = pd.read_csv("match_team_player.csv")
match_player = d.iloc[:, 106:]
#match_player.insert(0, "match_id", d.match_id)
#match_team.to_csv("match_team.csv", index = False)
match_player.to_csv("match_player.csv", index = False)

In [21]:
d.columns[106] # player starts from 106

'home_overall_rating_1'

In [152]:
# clean players
data = match_player
names = data.columns

to_drop = [x for x in names if x.find("preferred")!= -1 or x.find("birthday") != -1 or x.find('player_name') != -1 or x.find('weight') != -1]
data.drop(columns = to_drop, inplace = True) # Now each player has 39 attributes (37 from player attributes and height and weight)
names = data.columns

ordinals = [x for x in names if x.find("work_rate") != -1]

In [153]:
data.shape

(25979, 836)

In [151]:
# sort players
def row_combine_and_sort(row):
    '''
    New row will have (11-4)*2*2=28 fewer entries. From 836 to 808
    Attributes include:  36*11*2=814 (numerics), 2*4*2=16(ordinals)
    '''
    names = row.index
    num_features = 38
    ord_locs = [i for i,x in enumerate(names[:num_features]) if x.find("work_rate") != -1]
    
    
    def attribute_sort(start):
        index = [start + i * num_features for i in range(22)]
        home_values, away_values = row[index[:11]], row[index[11:]]
        home_values, away_values = home_values.to_list(), away_values.to_list()
        home_values.sort(reverse=True)
        away_values.sort(reverse=True)
        return home_values + away_values
    
    def ord_summary(nums):
        highs = sum([x=='high' for x in nums])
        mediums = sum([x=='medium' for x in nums])
        lows = sum([x=='low' for x in nums])
        missings = sum([x != x for x in nums])
        return [highs, mediums, lows, missings]
    
    def attribute_ord(start):
        index = [start + i * num_features for i in range(22)]
        home_values, away_values = row[index[:11]].to_list(), row[index[11:]].to_list()
        home_info = ord_summary(home_values)
        away_info = ord_summary(away_values)
        return home_info + away_info
    
    newrow = []
    row_names = []
    for i in range(num_features):
        if i not in ord_locs:
            newrow.extend(attribute_sort(i))
            col = names[i][:-1][4:]
            row_names.extend(['home' + col + str(i) for i in range(1,12,1)])
            row_names.extend(['away' + col + str(i) for i in range(1,12,1)])
        else:
            newrow.extend(attribute_ord(i))
            col = names[i][:-1][4:]
            row_names.extend(['home' + col + x for x in ['highs', 'mediums', 'lows', 'missings']])
            row_names.extend(['away' + col + x for x in ['highs', 'mediums', 'lows', 'missings']])
    
    assert len(newrow) == len(row_names)
    return pd.Series(newrow, index = row_names)

In [154]:
newrows = [row_combine_and_sort(row) for _,row in data.iterrows()]
newdata = pd.concat(newrows, axis = 1)
newdata = newdata.transpose()
data_players = newdata

In [155]:
data_players.shape # it should have 808 (reduced from 836) columns, including  36*11*2=792 (numerics), 2*4*2=16(ordinals)

(25979, 808)

In [156]:
data_players.to_csv("match_player_cleaned.csv", index = False)

In [157]:
# keep complete data
data_team = pd.read_csv("final_match_team.csv").drop(columns = ["season", "country_name", "league_name"])
data_team.shape

(25979, 29)

In [158]:
data_all = pd.concat([data_team.drop(columns = ["outcome"]), data_players], axis=1)
data_all.shape

(25979, 836)

In [159]:
rows_complete = np.mean(np.isnan(np.array(data_all)), axis = 1) == 0
np.sum(rows_complete)

16686

In [52]:
#match_dates = match_team.date.values
#later_times = [i for i,x in enumerate(match_dates) if type(x) != str or x >= "2010-02-22"]
#data_later = data_all.iloc[later_times, :]
#rows_complete = np.mean(np.isnan(np.array(data_later)), axis = 1) == 0

In [162]:
data_all = pd.concat([data_team, data_players], axis=1)
#data_later = data_all.iloc[later_times, :]
data_later_complete = data_all.iloc[rows_complete,:]
data_later_complete.shape

(16686, 837)

In [163]:
data_later_complete.to_csv("match_team_player_complete_after_20100222.csv", index = False)

In [141]:
# delete noisy features for gk.
def delete_non_gk(data):
    '''
    Reduce the number of features from 837 (29 + 36*11*2 + 2*4*2) to 253 (29+ 36*3*2 + 2 * 2 *2)
    '''
    
    gk_names = [x for x in data.columns if x.find("gk")!= -1 ]
    to_drop = [x for x in gk_names if x[-2:] != '_1']
    newdata  = data.drop(columns = to_drop)
    return newdata

In [164]:
data_later_complete_gkdropped = delete_non_gk(data_later_complete)
print(data_later_complete_gkdropped.shape)
data_later_complete_gkdropped.to_csv("match_team_player_complete_gkdropped.csv", index = False)

(16686, 737)


In [126]:
row = data_later_complete.iloc[0,:]

In [56]:
row.index[89]

'home_crossing_1'

In [57]:
row.index[29]

'home_overall_rating_1'

In [58]:
row.index[44+29]

'home_attacking_work_rate_highs'

In [59]:
# feature engineering: keep top 3
def summary_stats(nums):
    #return np.max(nums), np.min(nums), np.median(nums)
    return nums[0], nums[1], nums[2]

def summary_ords(nums):
    total = np.sum(nums[:3])
    #return list(nums[:3] / total) if total>0 else [np.nan] * 3
    return list(nums[:2] / total) if total>0 else [np.nan] * 2

def row_summary_stats(row):
    '''
    Reduce the number of features from 837 (29 + 36*11*2 + 2*4*2) to 253 (29+ 36*3*2 + 2 * 2 *2)
    '''
    newrow = list(row.values[:29])
    row_names = list(row.index[:29])
    # ords 
    ords_start = 44
    starts = [ords_start + x*4 for x in range(4)]
    starts = [x+29 for x in starts]
    for i in starts:
        newrow.extend(summary_ords(row.values[i:(i+4)]))
        row_names.extend(row.index[i:(i+2)])
    
    # numerics
    num_start_1 = 0
    num_start_2 = 60
    numerics = 36 - 2
    starts = [num_start_1 + x*11 for x in range(4)] + [num_start_2 + 11 * i for i in range(2*numerics)]
    starts = [x+29 for x in starts]
    for i in starts:
        newrow.extend(summary_stats(row.values[i:(i+11)]))
        t = row.index[i][:-2]
        #row_names.extend([t + x for x in ['_max', '_min', '_median']])
        row_names.extend([t + x for x in ['_1st', '_2nd', '_3rd']])
        
    assert len(newrow) == len(row_names)
    return pd.Series(newrow, index = row_names)

In [60]:
newrows = [row_summary_stats(row) for _,row in data_later_complete.iterrows()]
newdata = pd.concat(newrows, axis = 1)
newdata = newdata.transpose()
newdata.shape

(16686, 253)

In [61]:
newdata.columns[29:37]

Index(['home_attacking_work_rate_highs', 'home_attacking_work_rate_mediums',
       'away_attacking_work_rate_highs', 'away_attacking_work_rate_mediums',
       'home_defensive_work_rate_highs', 'home_defensive_work_rate_mediums',
       'away_defensive_work_rate_highs', 'away_defensive_work_rate_mediums'],
      dtype='object')

In [ ]:
#newdata.to_csv("datathon/match_team_player_narrow_after_20100222.csv", index = False)

In [62]:
# drop ordinal in players 
data_later_complete_narrow = newdata.drop(columns = newdata.columns[29:37]) # 253 - 8 = 245
data_later_complete_narrow.to_csv("match_team_player_narrow_complete_after_20100222.csv", index = False)

In [63]:
data_later_complete_narrow.shape

(16686, 245)

In [78]:
d.columns[31:63]

Index(['away_player_11', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA',
       'BSH', 'BSD', 'BSA', 'country_name'],
      dtype='object')

In [79]:
match_betting_odds = d.iloc[rows_complete,32:62]

In [81]:
match_betting_odds.shape

(16686, 30)

In [82]:
row = match_betting_odds.iloc[0,:]

In [102]:
def maximal_odds(row):
    newrow  = []
    for i in range(3):
        odds = row[np.arange(10) * 3 + i]
        if np.mean(np.isnan(odds)) == 1:
            newrow.append(np.nan)
        else:
            newrow.append(np.nanmax(odds))

    return pd.Series(newrow, index = ['max_odds_H', 'max_odds_D', 'max_odds_A'])

In [103]:
odds_rows = [maximal_odds(row) for _, row in match_betting_odds.iterrows()]
odds_rows = pd.concat(odds_rows, axis = 1)
odds_rows = odds_rows.transpose()

In [107]:
data_later_complete_narrow.shape

(16686, 245)

In [108]:
odds_rows.shape

(16686, 3)

In [111]:
data_later_with_odds = pd.concat([data_later_complete_narrow, odds_rows], axis = 1)
data_later_with_odds.shape

(16686, 248)

In [114]:
complete_rows_with_odds = data_later_with_odds.isnull().mean(axis = 1) == 0

In [117]:
data_later_with_odds = data_later_with_odds.loc[complete_rows_with_odds, :]
data_later_with_odds

In [119]:
data_later_with_odds.to_csv("match_team_player_narrow_complete_withodds.csv", index = False)

In [120]:
# keepers information
row = data_later_complete.iloc[0,:]

home_gk_positioning_7         21
home_gk_positioning_8         20
home_gk_positioning_9         20
home_gk_positioning_10        20
home_gk_positioning_11        20
away_gk_positioning_1         56
away_gk_positioning_2         23
away_gk_positioning_3         23
away_gk_positioning_4         23
away_gk_positioning_5         22
away_gk_positioning_6         21
away_gk_positioning_7         21
away_gk_positioning_8         21
away_gk_positioning_9         20
away_gk_positioning_10        20
away_gk_positioning_11        20
home_gk_reflexes_1            66
home_gk_reflexes_2            25
home_gk_reflexes_3            25
home_gk_reflexes_4            25
home_gk_reflexes_5            23
home_gk_reflexes_6            21
home_gk_reflexes_7            21
home_gk_reflexes_8            20
home_gk_reflexes_9            20
home_gk_reflexes_10           20
home_gk_reflexes_11           20
away_gk_reflexes_1            63
away_gk_reflexes_2            23
away_gk_reflexes_3            23
away_gk_re